In [1]:
!git clone https://github.com/saeedzou/InsulatorDetSeg.git
!git clone https://github.com/InsulatorData/InsulatorDataSet
!pip -q install ultralytics
from InsulatorDetSeg.utils import auto_generate_masks, convert_xywh_xyxy, generate_yolo_labels, get_unique_dims, make_train_valid
from ultralytics import SAM, YOLO
from ultralytics.models.sam import Predictor as SAMPredictor
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
%load_ext tensorboard

Cloning into 'InsulatorDetSeg'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 65 (delta 4), reused 7 (delta 2), pack-reused 53
Receiving objects: 100% (65/65), 54.38 MiB | 15.76 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Cloning into 'InsulatorDataSet'...
remote: Enumerating objects: 1964, done.
remote: Total 1964 (delta 0), reused 0 (delta 0), pack-reused 1964
Receiving objects: 100% (1964/1964), 387.92 MiB | 17.67 MiB/s, done.
Resolving deltas: 100% (1094/1094), done.
Updating files: 100% (1946/1946), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 8.5 MB/s eta 0:00:00


In [2]:
root_path = '/content/InsulatorDataSet'
generate_yolo_labels(root_path=root_path,
                     xml_path='Normal_Insulators/labels',
                     txt_path='Normal_Insulators/labels')
generate_yolo_labels(root_path=root_path,
                     xml_path='Defective_Insulators/labels/insulator',
                     txt_path='Defective_Insulators/labels')

In [3]:
make_train_valid(root='/content/InsulatorDataSet', val_size=0.2)

In [4]:
%%writefile CLIPD_v8.yaml
path: '/content/InsulatorDataSet/'
train: 'train/images'
val: 'valid/images'

# class names
names:
    0: 'insulator'

Writing CLIPD_v8.yaml


In [5]:
# Create SAMPredictor
overrides = dict(conf=0.3, task='segment', mode='predict', model="mobile_sam.pt", imgsz=1024, verbose=False)
predictor = SAMPredictor(overrides=overrides)

In [ ]:
HEIGHT = 864
WIDTH = 1152
auto_generate_masks(root='/content/InsulatorDataSet',
                    split='train',
                    sam_predictor=predictor,
                    height=HEIGHT,
                    width=WIDTH)
auto_generate_masks(root='/content/InsulatorDataSet',
                    split='valid',
                    sam_predictor=predictor,
                    height=HEIGHT,
                    width=WIDTH)

In [8]:
# train YOLO detection model
model = YOLO('yolov8m-seg.pt')
# train model for 5 epochs using CLIPD_v8.yaml dataset
model.train(data='/content/CLIPD_v8.yaml', epochs=100)

valid_results = model.val()
print(valid_results)

100%|██████████| 52.4M/52.4M [00:00<00:00, 312MB/s]
Ultralytics YOLOv8.0.157 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/content/CLIPD_v8.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, forma

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d05236b43a0>
fitness: 1.6905114530780525
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/precision(M)', 'metrics/recall(M)', 'metrics/mAP50(M)', 'metrics/mAP50-95(M)']
maps: array([      1.669])
names: {0: 'insulator'}
plot: True
results_dict: {'metrics/precision(B)': 0.9440077351731131, 'metrics/recall(B)': 0.9043473952142442, 'metrics/mAP50(B)': 0.9436529129582565, 'metrics/mAP50-95(B)': 0.8534478250663577, 'metrics/precision(M)': 0.9400082876854783, 'metrics/recall(M)': 0.9005159775897232, 'metrics/mAP50(M)': 0.9404066360722267, 'metrics/mAP50-95(M)': 0.8155582840169803, 'fitness': 1.6905114530780525}
save_dir: PosixPath('runs/segment/val')
seg: ultralytics.utils.metrics.Metric object
speed: {'preprocess': 1.95388398932